In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("D:/ETL HIVE/PYTHON/ADV PY/Cars93.csv")

In [5]:
df.isna().sum()

id                     0
Manufacturer           0
Model                  0
Type                   0
Min.Price              0
Price                  0
Max.Price              0
MPG.city               0
MPG.highway            0
AirBags               37
DriveTrain             0
Cylinders              0
EngineSize             0
Horsepower             0
RPM                    0
Rev.per.mile           0
Man.trans.avail        0
Fuel.tank.capacity     0
Passengers             0
Length                 0
Wheelbase              0
Width                  0
Turn.circle            0
Rear.seat.room         2
Luggage.room          11
Weight                 0
Origin                 0
Make                   0
dtype: int64

In [7]:
for i in df.columns:
    if df[i].dtypes==object:
        df[i]=df[i].fillna(df[i].mode()[0])
    else:
         df[i]=df[i].fillna(df[i].mean())
    
    

In [9]:
X = df.drop('Cylinders',axis=1)

In [11]:
Y = df[['Type']]

In [13]:
cat = []
con = []

for i in X.columns:
    if X[i].dtypes==object:
        cat.append(i)
    else:
        con.append(i)

In [15]:
Xcat = X [cat]
Xcon = X [con]

In [17]:
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [19]:
le = LabelEncoder()
ss = StandardScaler()

In [21]:
for i in Xcat.columns:
    Xcat[i]=le.fit_transform(Xcat[i])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19356\188807692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xcat[i]=le.fit_transform(Xcat[i])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19356\188807692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xcat[i]=le.fit_transform(Xcat[i])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_19356\188807692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [23]:
Xcon = pd.DataFrame(ss.fit_transform(Xcon),columns=con)

In [25]:
X = Xcon.join(Xcat)

In [27]:
out = []
for i in Xcon.columns:
    o = Xcon[(Xcon[i]<-3)|(Xcon[i]>3)].index
    out.extend(o)
out

[47, 58, 58, 58, 38, 41, 38, 41, 16, 59]

In [29]:
a = list(set(out))
a

[38, 41, 47, 16, 58, 59]

In [31]:
X = X.drop(index=a,axis=0)
Y = Y.drop(index=a,axis=0)

In [33]:
X.index=range(X.shape[0])
Y.index=range(Y.shape[0])

In [35]:
from sklearn.model_selection import train_test_split

In [37]:
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=42)

In [39]:
from sklearn.linear_model import LogisticRegression

In [41]:
lr = LogisticRegression()

In [43]:
model = lr.fit(xtrain,ytrain)

C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
tr_pred=model.predict(xtrain)
ts_pred=model.predict(xtest)

In [47]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [49]:
tr_score = accuracy_score(ytrain,tr_pred)
tr_score

1.0

In [51]:
ts_score = accuracy_score(ytest,ts_pred)
ts_score

0.8333333333333334

In [53]:
tr_conf = confusion_matrix(ytrain,tr_pred)
ts_conf = confusion_matrix(ytest,ts_pred)
tr_conf

array([[12,  0,  0,  0,  0,  0],
       [ 0,  9,  0,  0,  0,  0],
       [ 0,  0, 17,  0,  0,  0],
       [ 0,  0,  0, 16,  0,  0],
       [ 0,  0,  0,  0,  8,  0],
       [ 0,  0,  0,  0,  0,  7]], dtype=int64)

In [55]:
ts_conf

array([[4, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0],
       [0, 1, 2, 0, 0, 0],
       [0, 0, 0, 1, 2, 0],
       [0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 0, 1]], dtype=int64)